In [60]:
import casadi as ca
import cyecca.lie as lie


In [61]:
def derive_model():
    # p, parameters
    thr_max = ca.SX.sym("thr_max")
    m = ca.SX.sym("m")
    cl = ca.SX.sym("cl")
    cd = ca.SX.sym("cd")
    S = ca.SX.sym("S")
    rho = ca.SX.sym("rho")
    g = ca.SX.sym("g")

    p= ca.vertcat(
        thr_max,
        m,
        cl,
        cd,
        S,
        rho,
        g
    )
    p_defaults = {
        "thr_max" : 1.0,
        "m" : 0.2,
        "cl": 3.5,
        "cd" : 0.0,
        "S":1.0,
        "rho": 1.225,
        "g": 9.8,
    }

    # states
    # x, state
    posx = ca.SX.sym("posx")
    posy = ca.SX.sym("posy")
    posz = ca.SX.sym("posz")
    velx = ca.SX.sym("velx")
    vely = ca.SX.sym("vely")
    velz = ca.SX.sym("velz")
    x = ca.vertcat(
        posx,
        posy,
        posz,
        velx,
        vely,
        velz,
        )
    x0_defaults = {
        "posx" : 0,
        "posy" : 0,
        "posz" : 0,
        "velx" : 0,
        "vely" : 0,
        "velz" : 0,
    }

    # input
    throttle_cmd = ca.SX.sym("throttle_cmd")
    u = ca.vertcat(throttle_cmd)


    # force and moment
    VT = ca.norm_2(ca.vertcat(velx,vely,velz))
    q = 0.5 * rho * VT **2

    # velx = (ax_b)*dt
    ground = ca.if_else( posz<0,
                        -posz * 15000 - velz * 15000,
                        0)
    D = cd * q * S
    L = cl * q * S

    fx_b = (thr_max*u[0]-velx) - D
    ax_b = fx_b/m
    fz_b = L - m *g +ground
    az_b = fz_b/m
    # velz = az_b *dt


    # states derivative
    posx_dot = velx #ax_b * dt
    posy_dot = vely
    posz_dot = velz #az_b *dt
    velx_dot = ax_b
    vely_dot = 0 #assume no slip
    velz_dot = az_b
    xdot = ca.vertcat(posx_dot, posy_dot, posz_dot, velx_dot, vely_dot, velz_dot)


    # algebraic (these algebraic expressions are used during the simulation)
    z = ca.vertcat()
    alg = z


    f = ca.Function("f", [x, u, p], [xdot], ["x", "u", "p"], ["xdot"])

    dae = {"x": x, "ode": f(x, u, p), "p": p, "u": u, "z": z, "alg": alg}

    return locals()


In [62]:
pnew = [1, 0.1, 1.8, 1, 0, 1.225, 9.8] # [m cl S] 
unew = [1] #throttle

model = derive_model()


In [63]:
F = ca.integrator('F', 'idas', model["dae"])
res = F(x0=[0,0,0,0,0,0], z0=0, u=[1], p=pnew)
res["xf"]

DM([0.900005, -2.34e-23, -4.12985e-05, 0.999954, -2.34004e-23, -2.4035e-05])

In [64]:
F = ca.integrator('F', 'idas', model["dae"], 0,1)
res = F(x0=[0,0,0,6,0,2], z0=0, u=[1], p=pnew)
res["xf"]

DM([1.49998, 3.74014e-21, -3.65612e-05, 1.00023, 6.32056e-21, -2.87724e-05])

In [65]:
type(res["xf"])

casadi.casadi.DM

In [66]:
import numpy as np
np.array(res["xf"]).reshape(-1)

array([ 1.49997716e+00,  3.74013776e-21, -3.65611610e-05,  1.00022843e+00,
        6.32055546e-21, -2.87723642e-05])

In [101]:
def derive_model():
    # p, parameters
    thr_max = ca.SX.sym("thr_max")
    m = ca.SX.sym("m")
    cl = ca.SX.sym("cl")
    cd = ca.SX.sym("cd")
    S = ca.SX.sym("S")
    rho = ca.SX.sym("rho")
    g = ca.SX.sym("g")
    Jx = ca.SX.sym("Jx")
    Jy = ca.SX.sym("Jy")
    Jz = ca.SX.sym("Jz")
    J = ca.diag(ca.vertcat(Jx, Jy, Jz))
    Cl_p = ca.SX.sym("Cl_p")
    Cm_q = ca.SX.sym("Cm_q")
    Cn_r = ca.SX.sym("Cn_r")
    p= ca.vertcat(
        thr_max,
        m,
        cl,
        cd,
        S,
        rho,
        g,
        Jx,
        Jy,
        Jz,
        Cl_p,
        Cm_q,
        Cn_r,
    )
    p_defaults = {
        "thr_max" : 1.0,
        "m" : 0.2,
        "cl": 3.5,
        "cd" : 0.0,
        "S":1.0,
        "rho": 1.225,
        "g": 9.8,
        'Jx': 0.0217,
        'Jy': 0.0217,
        "Jz" : 0.04,
        "Cl_p": 0,
        "Cm_q": 0,
        "Cn_r": 0,
    }

    # states
    # # x, state
    # posx = ca.SX.sym("posx")
    # posy = ca.SX.sym("posy")
    # posz = ca.SX.sym("posz")
    # velx = ca.SX.sym("velx")
    # vely = ca.SX.sym("vely")
    # velz = ca.SX.sym("velz")

    position_w = ca.SX.sym("position_w",3) # w = world frame
    velocity_b = ca.SX.sym("velocity_b",3)
    quat_wb = ca.SX.sym("quat_b",4) # Quaternion world - body frame
    # quad_w = ca.SX.sym("quad_w",4)
    omega_wb_b = ca.SX.sym("omega_wb_b",3)
    # quat_bn = ca.SX.sym("quat_bn",4) # Quaternion body - wind frame
    # omega_bn_b = ca.SX.sym("omega_bn", 3) #omega body-wind

    x = ca.vertcat(
        position_w,
        velocity_b,
        quat_wb,
        omega_wb_b,
        # quat_bn,
        # omega_bn_b ,
        )
    x0_defaults = {
        "position_w_0" : 0,
        "position_w_1" : 0,
        "position_w_2" : 0,
        "velocity_b_0" : 0.001,
        "velocity_b_1" : 0.001,
        "velocity_b_2" : 0.001,
        "quat_wb_0" : 1,
        "quat_wb_1" : 0,
        "quat_wb_2" : 0,
        "quat_wb_3" : 0,
        "omega_wb_b_0" : 0,
        "omega_wb_b_1" : 0,
        "omega_wb_b_2" : 0,
        # "quat_bn_0" : 0,
        # "quat_bn_1" : 0,
        # "quat_bn_2" : 0,
        # "quat_bn_3" : 0,
        # "omega_bn_b_0" : 0,
        # "omega_bn_b_1" : 0,
        # "omega_bn_b_2" : 0,
    }

    # input
    throttle_cmd = ca.SX.sym("throttle_cmd")
    elev_cmd = ca.SX.sym("elev_cmd")

    u = ca.vertcat(throttle_cmd, elev_cmd)


    # Defining frames
    # code this, recheck in stewen and lewis
   # alpha = atan(vel_b_z/vel_b_x)
   # beta = y/atan(posy/posx)
   # test =lie.SO3EulerB321.elem(ca.vertcat(beta,-alpha,0))
    # q_bn = lie.SO3Quat.from_Euler(test)  #body to wind
    # fn = ca.vert(L ,sideforce,drag) #n is wind frame
    # fb= q_bn@fn #force in body frame

    # idea:calculate alpha and beta based on stephen and lewis
    # quat --> world to body
    # Quat --> body to wind
    xAxis = ca.vertcat(1, 0, 0)
    yAxis = ca.vertcat(0, 1, 0)
    zAxis = ca.vertcat(0, 0, 1)


    # VT = ca.norm_2(ca.vertcat(velocity_b[0],velocity_b[1],velocity_b[2]))
    V_b = ca.norm_2(velocity_b)
    alpha = ca.atan(velocity_b[2]/velocity_b[0])
    beta = ca.asin(velocity_b[1]/V_b) #RECHECK
    euler_n = lie.SO3EulerB321.elem(ca.vertcat(beta, -alpha, 0)) # Euler elements for wind frame
    quat_bn = lie.SO3Quat.from_Euler(euler_n)

    quat_wb = lie.SO3Quat.elem(quat_wb)
    quat_bw = quat_wb.inverse()
    P = omega_wb_b[0]
    Q = omega_wb_b[1]
    R = omega_wb_b[2]


    # force and moment
    qbar = 0.5 * rho * velocity_b[0]**2# VT **2 # TODO velocity should be in wind frame

    # velx = (ax_b)*dt
    ground = ca.if_else(position_w[2]<0,
                        -position_w[2] * 150 - velocity_b[2] * 150,
                        0)
    D = cd * qbar * S
    L = cl * qbar * S

    fx_b = (thr_max*u[0]-velocity_b[0]) - D
    ax_b = fx_b/m
    fz_b = L - m *g +ground
    az_b = fz_b/m

    Fs = 0 #side force
    F_n = ca.vertcat(L, Fs, D) #force in wind frame (n)
    F_b = quat_bn@F_n # Aerodynamic force from wind

    F_b += quat_bw @ (-m * g * zAxis) # gravity

    # #Angle
    # e = SO3EulerB321.elem(ca.SX.sym("e",3))
    # qw = SO3Quat.from_Euler(e).param #param gets array



    # Moment
    M_b = ca.vertcat(0, 0, 0)
    Cl = Cl_p * P  # rolling moment
    Cm = Cm_q * Q  # pitching moment
    Cn = Cn_r * R  # yawing moment
    Fi_b = fx_b * xAxis #thrust
    Mi_b = ca.vertcat(Cl, Cm, Cn) * S # aerodynamic moment in body frame
    F_b += Fi_b
    M_b += Mi_b

    # # kinematics TODO ADD THIS
    derivative_omega_wb_b = ca.inv(J) @ (M_b - ca.cross(omega_wb_b, J @ omega_wb_b))
    derivative_quaternion_wb = quat_wb.right_jacobian() @ omega_wb_b
    derivative_position_w = quat_wb @ velocity_b
    derivative_velocity_b = F_b / m - ca.cross(omega_wb_b, velocity_b)
    # derivative_omega_bn_b = ca.inv(J) @ (M_b - ca.cross(omega_bn_b, J @ omega_bn_b))
    # derivative_quaternion_bn = quat_bn.right_jacobian() @ omega_bn_b




    # # states derivative
    # posx_dot = velocity_b[0] #ax_b * dt
    # posy_dot = velocity_b[1]
    # posz_dot = velocity_b[2] #az_b *dt
    # velx_dot = ax_b
    # vely_dot = 0 #assume no slip
    # velz_dot = az_b
    # xdot = ca.vertcat(posx_dot, posy_dot, posz_dot, velx_dot, vely_dot, velz_dot)

    # state derivative vector
    xdot = ca.vertcat(
        derivative_position_w,
        derivative_velocity_b,
        derivative_quaternion_wb,
        derivative_omega_wb_b,
        # derivative_quaternion_bn,
        # derivative_omega_bn_b,
    )


    # algebraic (these algebraic expressions are used during the simulation)
    z = ca.vertcat()
    alg = z


    f = ca.Function("f", [x, u, p], [xdot], ["x", "u", "p"], ["xdot"])

    dae = {"x": x, "ode": f(x, u, p), "p": p, "u": u, "z": z, "alg": alg}

    return locals()

In [102]:
model = derive_model()
x0_dict = model["x0_defaults"]
state = np.array(list(x0_dict.values()),dtype=float)
p_dict = model["p_defaults"]
p = np.array(list(p_dict.values()), dtype=float)

In [103]:
state
p

array([1.    , 0.2   , 3.5   , 0.    , 1.    , 1.225 , 9.8   , 0.0217,
       0.0217, 0.04  , 0.    , 0.    , 0.    ])

In [104]:
# model["F_b"]

In [105]:
unew = [1,0] #throttle

F = ca.integrator('F', 'idas', model["dae"], 0, 0.1)
res = F(x0=state, z0=0, u=[0.6,0.6], p=p)
res["xf"]

DM([0.00100304, 0.00100206, 0.00100367, 0.00898664, 0.0110795, 0.00999399, 0.984823, 0.110405, 0.118448, 0.0624931, 4.27121, 4.91809, 2.5])

In [98]:
F

Function(F:(x0[13],z0[0],p[13],u[2],adj_xf[],adj_zf[],adj_qf[])->(xf[13],zf[0],qf[0],adj_x0[],adj_z0[],adj_p[],adj_u[]) IdasInterface)

In [84]:
F(x0=state, z0=0, p=p, u=ca.vertcat(0,0))


CasADi - 2024-10-24 13:27:17 WARNING("F:daeF failed: NaN detected for output ode, at (row 3, col 0).") [.../casadi/core/oracle_function.cpp:393]
The residual function failed at the first call. 


RuntimeError: Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:1432:
Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:361:
.../casadi/interfaces/sundials/idas_interface.cpp:596: IDACalcIC returned "IDA_FIRST_RES_FAIL". Consult IDAS documentation.